# Import library

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import os
import cv2
from data_helper import load_features, load_labels, __model_file__, split_train_test_valid
from modeling import get_training_model, CheckpointCallback
from run_training_encoders import run_training, run_training_encoders
from matplotlib import pyplot as plt
from collections import Counter
import sys

# Set up TPUs or GPUs

In [ ]:
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
# Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
  strategy = tf.distribute.experimental.TPUStrategy(tpu) 
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

# Training

In [ ]:
# Example: combination_path = vein_texture_fourier_xyprojection_path = 'features_data/vein_texture_fourier_xyprojection_feature.csv'

args = sys.argv
run_training_encoders(combination_path)